<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/src/Token_head_importance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# import sys
# sys.path.append('/content/drive/My Drive/{}'.format("cogs402longformer/"))

In [3]:
pip install datasets --quiet

In [4]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import Dataset and Model

In [5]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

In [6]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096')

def longformer_finetuned_papers():
    model = AutoModelForSequenceClassification.from_pretrained('danielhou13/longformer-finetuned_papers', num_labels = 2, output_attentions = True)
    return model

def preprocess_function(tokenizer, example, max_length):
    example.update(tokenizer(example['text'], padding='max_length', max_length=max_length, truncation=True))
    return example

def get_papers_dataset(dataset_type):
    max_length = 2048
    dataset = load_dataset("danielhou13/cogs402dataset")[dataset_type]

    # tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    dataset = dataset.map(lambda x: preprocess_function(tokenizer, x, max_length), batched=True)
    setattr(dataset, 'input_columns', ['input_ids', 'attention_mask'])
    setattr(dataset, 'target_columns', ['labels'])
    setattr(dataset, 'max_length', max_length)
    setattr(dataset, 'tokenizer', tokenizer)
    return dataset

def papers_test_set():
    return get_papers_dataset('test')

In [7]:
cogs402_test = papers_test_set()
model = longformer_finetuned_papers()
columns = cogs402_test.input_columns + cogs402_test.target_columns
print(columns)
cogs402_test.set_format(type='torch', columns=columns)
cogs402_test=cogs402_test.remove_columns(['text'])

Using custom data configuration danielhou13--cogs402dataset-cc784554b797f843
Reusing dataset parquet (/root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402dataset-cc784554b797f843/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/2 [00:00<?, ?it/s]

Parameter 'function'=<function get_papers_dataset.<locals>.<lambda> at 0x7fce9968eef0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

['input_ids', 'attention_mask', 'labels']


In [8]:
if torch.cuda.is_available():
    model = model.cuda()

print(model.device)

cuda:0


Take example for evaluation

In [9]:
testexam = cogs402_test[923]

In [10]:
# print(test['labels'][923])

In [11]:
output = model(testexam["input_ids"].unsqueeze(0).cuda(), attention_mask=testexam['attention_mask'].unsqueeze(0).cuda(), labels=testexam['labels'].cuda())
batch_attn = output[-2]
output_attentions = torch.stack(batch_attn).cpu()
global_attention = output[-1]
output_global_attentions = torch.stack(global_attention).cpu()
print("output_attention.shape", output_attentions.shape)
print("gl_output_attention.shape", output_global_attentions.shape)

output_attention.shape torch.Size([12, 1, 12, 2048, 514])
gloutput_attention.shape torch.Size([12, 1, 12, 2048, 1])


In [12]:
# print(os.getcwd())
# yes = torch.load("resources/longformer_test2/epoch_3/aggregate_attn.pt")

Convert sliding attention matrix to correct seq_len x seq_len matrix

In [13]:
def create_head_matrix(output_attentions, global_attentions):
    new_attention_matrix = torch.zeros((output_attentions.shape[0], 
                                      output_attentions.shape[0]))
    for i in range(output_attentions.shape[0]):
        test_non_zeroes = torch.nonzero(output_attentions[i]).squeeze()
        test2 = output_attentions[i][test_non_zeroes[1:]]
        new_attention_matrix_indices = test_non_zeroes[1:]-257 + i
        new_attention_matrix[i][new_attention_matrix_indices] = test2
        new_attention_matrix[i][0] = output_attentions[i][0]
        new_attention_matrix[0] = global_attentions.squeeze()[:output_attentions.shape[0]]
    return new_attention_matrix


def attentions_all_heads(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = create_head_matrix(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

def all_batches(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = attentions_all_heads(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

def all_layers(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = all_batches(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

In [14]:
converted_mat = all_layers(output_attentions, output_global_attentions).detach().cpu().numpy()
print(converted_mat.shape)

(12, 1, 12, 2048, 2048)


Sum over all the tokens (column-wise)

In [28]:
attention_sum = converted_mat.sum(axis=3)
print(attention_sum.shape)

(12, 1, 12, 2048)


Load head importance model and scale the attentions by head importance

In [16]:
head_importance = torch.load("/content/drive/MyDrive/cogs402longformer/t3-visapplication/resources/pretrained/head_importance.pt")

In [17]:
print(head_importance[2])

[0.00404374 0.02163236 0.00531127 0.01472499 0.03868961 0.00646301
 0.01300004 0.00334545 1.         0.00796655 0.1583804  0.01080081]


In [18]:
def scale_by_importance(attention_matrix, head_importance):
  new_matrix = np.zeros_like(attention_matrix)
  for i in range(attention_matrix.shape[0]):
    head_importance_layer = head_importance[i]
    for j in range(attention_matrix.shape[1]):
      new_matrix[i][j] = attention_matrix[i][j] * np.expand_dims(head_importance_layer, axis=1)
  return new_matrix

In [32]:
attention_matrix_importance = scale_by_importance(attention_sum, head_importance)
print(attention_matrix_importance.shape)

(12, 1, 12, 2048)


Get top k attended words for each head, for each example in batch, for each layer

In [34]:
def find_top_attention(scores_mat, axis, k):
  indices = np.argpartition(scores_mat, -k, axis=axis)[:, :, :, -k:]
  print(indices.shape)
  vals = np.take_along_axis(scores_mat, indices, axis=axis)
  return indices, vals

In [21]:
all_tokens = tokenizer.convert_ids_to_tokens(testexam["input_ids"])

In [35]:
indexes, values = find_top_attention(attention_matrix_importance, 3, 4)
print(indexes.shape)

(12, 1, 12, 4)
(12, 1, 12, 4)


In [37]:
def get_tokens(index_matrix):
  highest_tokens = []
  for i in range(indexes.shape[0]):
    row_tokens = []
    for j in range(indexes.shape[1]):
      batch_tokens = []
      for k in range(indexes.shape[2]):
        tokens = [all_tokens[idx] for idx in indexes[i][j][k]]
        batch_tokens.append(tokens)
      row_tokens.append(batch_tokens)
    highest_tokens.append(row_tokens)
  return np.array(highest_tokens)

highest_tokens = get_tokens(indexes)
print(highest_tokens.shape)

(12, 1, 12, 4)


In [38]:
print(indexes[:][:][0], values[:][:][0], highest_tokens[:][:][0])

[[[1024 1536  512    0]
  [ 195 1024 1536    0]
  [ 848  512 1024    0]
  [1852 1024  735    0]
  [1074 1480 1145    0]
  [1024 1536  512    0]
  [1536 1024  512    0]
  [ 756  587 1706  663]
  [ 195 1677 1701    0]
  [ 640 1721 1222  570]
  [ 570  512 1024    0]
  [1316  569 1222    0]]] [[[ 0.16710556  0.18387426  0.42733374  1.0553689 ]
  [ 0.3001202   0.34594324  0.40609905  0.9839709 ]
  [ 0.16722995  0.18950084  0.20314223  0.9272709 ]
  [ 0.13849676  0.15325466  0.1563048   0.62078804]
  [ 0.6675363   0.69763726  0.7251215   2.0927174 ]
  [ 0.58266914  0.6524942   2.1246636  11.776898  ]
  [ 0.19867428  0.2547237   0.31990826  1.6099393 ]
  [ 0.45979688  0.46923503  0.5016827   0.5077759 ]
  [ 0.25108713  0.30320972  0.2798093   0.54474884]
  [ 0.08754832  0.09248763  0.14845492  0.11433721]
  [ 0.14119865  0.14136115  0.1439912   0.82171977]
  [ 0.08574928  0.08625139  0.12115718  0.1942717 ]]] [[['Ġoptimizations' 'Ġconstants' '(' '<s>']
  ['Ġsyntax' 'Ġoptimizations' 'Ġconstant